In [13]:
#step 2: clean the data because even though it's regarding the highest office in the country,
#it's still a govennment website

import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

df_old = pd.read_csv('scraped_data.csv')
print(df_old.shape)
print(df_old.head())

(10898, 1)
                                                data
0  Executive Order 7532 \nEstablishing Shinnecock...
1  Signed: January 8, 1937\nFederal Register page...
2  Executive Order   7533 \nExemption of James E....
3  Signed: January 11, 1937 \nNote: This Executiv...
4  Executive Order   7534 \nExcluding Land From S...


In [14]:
#This is the ugly part where I have to manually edit some parts of the data
#pls forgive me

#just to see the rows of data that need manual edits
df_bads = df_old[~df_old['data'].str.startswith('Executive')]
df_bads = df_bads[~df_bads['data'].str.startswith('Signed')]
print(df_bads.shape)
print(df_bads)

(6, 1)
                                                    data
9582   Decontrol of crude oil and refined petroleum p...
9808          President's Commission on Strategic Forces
10004                        Regulatory planning process
10094  Prohibiting trade and certain transactions inv...
10254  Delegating authority to implement assistance t...
10396  Visit the Ronald Reagan Presidential Library a...


In [15]:
#lil ugly manual adjustments to insert missing order numbers
#row 9582 - order number 12287 - 1981
df_old.loc[[9582]] = df_old.loc[[9582]].replace('Decontrol of crude oil and refined petroleum products', 'Executive Order 12287 \nDecontrol of crude oil and refined petroleum products', regex=False)

#row 9808 - order number 12400 - 1983
df_old.loc[[9808]] = df_old.loc[[9808]].replace('President\'s Commission on Strategic Forces', 'Executive Order 12400 \nPresident\'s Commission on Strategic Forces', regex=False)

#row 10004 - order number 12498 - 1985
df_old.loc[[10004]] = df_old.loc[[10004]].replace('Regulatory planning process', 'Executive Order 12498 \nRegulatory planning process', regex=False)

#row 10094 - order number 12543 - 1986
df_old.loc[[10094]] = df_old.loc[[10094]].replace('Prohibiting trade and certain transactions involving Libya', 'Executive Order 12543 \nProhibiting trade and certain transactions involving Libya', regex=False)

#row 10254 - order number 12623 - 1988
df_old.loc[[10254]] = df_old.loc[[10254]].replace('Delegating authority to implement assistance to the Nicaraguan Democratic Resistance', 'Executive Order 12623 \nDelegating authority to implement assistance to the Nicaraguan Democratic Resistance', regex=False)

#row 10396 - order number 12663 - 1989
df_old.loc[[10396]] = df_old.loc[[10396]].replace('Visit the Ronald Reagan Presidential Library and Museum', 'Executive Order 12663 \nAdjustments of certain rates of pay and allowances', regex=False)

#fixing formatting/typos of a few signing dates
#row 2145 - signing date and fed register page and date are mashed together
df_old.loc[[2145]] = df_old.loc[[2145]].replace('Signed: 5 FR 4558, November 18, 194November 20, 1940', 'Signed: November 18, 1940 \nFederal Register page and date: 5 FR 4558, November 20, 1940', regex=True)

#row 1533 - insert '\n' after signing date
df_old.loc[[1533]] = df_old.loc[[1533]].replace('Signed: November 30, 1939 ', 'Signed: November 30, 1939 \n', regex=True)

#row 5201 - change year from 950 to 1950
df_old.loc[[5201]] = df_old.loc[[5201]].replace('Signed: January 31, 950', 'Signed: January 31, 1950', regex=True)

#row 6243 - insert space between comma and year
df_old.loc[[6243]] = df_old.loc[[6243]].replace('Signed: June 29,1955', 'Signed: June 29, 1955', regex=True)

#row 6367 - correct year from 19656 to 1956
df_old.loc[[6367]] = df_old.loc[[6367]].replace('Signed: October 22, 19656', 'Signed: October 22, 1956', regex=True)

#row 4871 - correct year from 1945 to 1948
df_old.loc[[4871]] = df_old.loc[[4871]].replace('Signed: March 22, 1945', 'Signed: March 22, 1948', regex=True)

#row 4563 - correct year from 1948 to 1946
df_old.loc[[4563]] = df_old.loc[[4563]].replace('Signed: October 5, 1948', 'Signed: October 5, 1946', regex=True)

#row 2893 - correct year from 1946 to 1941
df_old.loc[[2893]] = df_old.loc[[2893]].replace('Signed: December 11, 1946', 'Signed: December 11, 1941', regex=True)

#row 5551 - correct year from 1961 to 1951
df_old.loc[[5551]] = df_old.loc[[5551]].replace('Signed: July 31, 1961', 'Signed: July 31, 1951', regex=True)

#row 5551 - correct date from July 6 to July 26
df_old.loc[[2619]] = df_old.loc[[2619]].replace('Signed: July 6, 1941', 'Signed: July 26, 1941', regex=True)

#row 5551 - correct date from August 1 to August 11
df_old.loc[[5301]] = df_old.loc[[5301]].replace('Signed: August 1, 1950', 'Signed: August 11, 1950', regex=True)

In [12]:
print(df_old.loc[[5301]])

                                                   data
5301  Signed: August 1, 1950\nFederal Register page ...


In [16]:
#to view and verify they updated properly
print(df_old.loc[[9582]])
print(df_old.loc[[9808]])
print(df_old.loc[[10004]])
print(df_old.loc[[10094]])
print(df_old.loc[[10254]])
print(df_old.loc[[10396]])
print(df_old.loc[[2145]])
print(df_old.loc[[1533]])
print(df_old.loc[[5201]])
print(df_old.loc[[6243]])
print(df_old.loc[[6367]])
print(df_old.loc[[4871]])
print(df_old.loc[[4563]])
print(df_old.loc[[2893]])
print(df_old.loc[[5551]])
print(df_old.loc[[2619]])
print(df_old.loc[[5301]])

                                                   data
9582  Executive Order 12287 \nDecontrol of crude oil...
                                                   data
9808  Executive Order 12400 \nPresident's Commission...
                                                    data
10004  Executive Order 12498 \nRegulatory planning pr...
                                                    data
10094  Executive Order 12543 \nProhibiting trade and ...
                                                    data
10254  Executive Order 12623 \nDelegating authority t...
                                                    data
10396  Executive Order 12663 \nAdjustments of certain...
                                                   data
2145  Signed: November 18, 1940 \nFederal Register p...
                                                   data
1533  Signed: November 30, 1939 \nFederal Register p...
                                                   data
5201  Signed: January 31, 1950\nFederal 

In [17]:
#split the dataframe and make sure they're of equal length
df_nums = df_old[df_old['data'].str.startswith('Executive')]
df_dates = df_old[df_old['data'].str.startswith('Signed')]
print(df_nums.shape)
print(df_dates.shape)

(5449, 1)
(5449, 1)


In [18]:
#trim unecessary strings and make for easier processing
#because some rows have '\n' between 'Executive Order' and the order number
df_nums['data'] = df_nums['data'].str.replace('Executive Order', '', regex=False)
df_nums['data'] = df_nums['data'].str.replace('No.', '', regex=False)
df_nums['data'] = df_nums['data'].str.strip()
#split the order number and title
df_nums[['executive_order_number', 'title']] = df_nums['data'].str.split('\n', n=1, expand=True)

print(df_nums.head())

df_dates['data'] = df_dates['data'].str.replace('Signed:', '', regex=False)
df_dates['data'] = df_dates['data'].str.strip()
#split signing date and disposition notes
df_dates[['signing_date', 'disposition_notes']] = df_dates['data'].str.split('\n', n=1, expand=True)

print(df_dates.head())

                                                data executive_order_number  \
0  7532 \nEstablishing Shinnecock Migratory Bird ...                  7532    
2  7533 \nExemption of James E. Tibbitts From Com...                  7533    
4  7534 \nExcluding Land From Sitgreaves National...                  7534    
6  7535 \nPartial Revocation of   6110 of May 2, ...                  7535    
8  7536 \nPartial Revocation of   5687 of August ...                  7536    

                                               title  
0  Establishing Shinnecock Migratory Bird Refuge,...  
2  Exemption of James E. Tibbitts From Compulsory...  
4  Excluding Land From Sitgreaves National Forest...  
6  Partial Revocation of   6110 of May 2, 1933, W...  
8  Partial Revocation of   5687 of August 18, 193...  
                                                data       signing_date  \
1  January 8, 1937\nFederal Register page and dat...    January 8, 1937   
3  January 11, 1937 \nNote: This Executive or

In [19]:
df_nums['count'] = [*range(df_nums.shape[0])]
df_dates['count'] = [*range(df_dates.shape[0])]
df_all = df_nums[['executive_order_number', 'title', 'count']].merge(df_dates[['signing_date', 'disposition_notes', 'count']], on='count')
df_all = df_all.drop(['count'], axis = 1)

for col in list(df_all):
    df_all[col] = df_all[col].str.strip()

print(df_all.head())

  executive_order_number                                              title  \
0                   7532  Establishing Shinnecock Migratory Bird Refuge,...   
1                   7533  Exemption of James E. Tibbitts From Compulsory...   
2                   7534  Excluding Land From Sitgreaves National Forest...   
3                   7535  Partial Revocation of   6110 of May 2, 1933, W...   
4                   7536  Partial Revocation of   5687 of August 18, 193...   

       signing_date                                  disposition_notes  
0   January 8, 1937  Federal Register page and date: 2 FR 63, Janua...  
1  January 11, 1937  Note: This Executive order was not received fo...  
2  January 12, 1937  Federal Register page and date: 2 FR 87, Janua...  
3  January 12, 1937  Federal Register page and date: 2 FR 88, Janua...  
4  January 12, 1937  Federal Register page and date: 2 FR 88, Janua...  


In [20]:
df_all['executive_order_number'] = df_all['executive_order_number'].str.replace('-A', '.2', regex = False)
df_all['executive_order_number'] = df_all['executive_order_number'].str.replace('A', '.5', regex = False)
df_all['executive_order_number'] = df_all['executive_order_number'].str.replace('-B', '.5', regex = False)
df_all['executive_order_number'] = df_all['executive_order_number'].str.replace('-C', '.7', regex = False)
df_all['executive_order_number'] = df_all['executive_order_number'].astype(float)
df_all.sort_values(by='executive_order_number', ascending=False, inplace = True)

In [21]:
df_all.to_csv('cleaned_data.csv', index=False)